In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from random import normalvariate
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import find_peaks
from scipy import fftpack,signal
from scipy.linalg import expm
from scipy.linalg import logm
from scipy.special import jv
from scipy import interpolate, integrate
from scipy.integrate import complex_ode,simpson
from scipy import special
from scipy.optimize import curve_fit
from sympy.physics.wigner import wigner_3j ,wigner_6j
#from maxwellbloch import mb_solve, field, ob_atom
import matplotlib as mpl
from scipy.integrate import quad

In [30]:
# ref: Efficient algorithm to compute the second Chern number in four dimensional systems
def testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta):
    sinteta = B/2/np.pi/beta
    costeta = np.sqrt(1-sinteta**2)
    arg0 = 2* np.sin(np.pi * beta * sinteta)
    arg1 = 2 * np.sin(np.pi * beta * costeta)
    J0 = jv(0,arg0)
    J1 = jv(1,arg1) 
    sigmax = np.array([[0,1],[1,0]])
    sigmay = np.array([[0,-1j],[1j,0]])
    sigmaz = np.array([[1,0],[0,-1]])

    gamma0 = np.kron(sigmax,np.eye(2))
    gamma1 = np.kron(sigmay,np.eye(2))
    gamma2 = np.kron(sigmaz,sigmax)
    gamma3 = np.kron(sigmaz,sigmay)
    gamma4 = np.kron(sigmaz,sigmaz)

    h4 = mz0-2*t0*J0*np.cos(kx)-2*t0*J1*np.cos(ky)-2*tz*np.cos(kz)-mz*np.cos(kw)
    h1 = -2*tso*J0*np.sin(kx)
    h2 = 2*tso*J1*np.sin(ky)
    h3 = -2*tsoz*np.sin(kz)
    h0 = ton*np.sin(kw)
    H = h0*gamma0+h1*gamma1+h2*gamma2+h3*gamma3+h4*gamma4
    return H

def nonAbeianUxy(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    
    H = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta)
    Hdkx =testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx+dkx,ky,kz,kw,B,beta)
    Hdky = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky+dky,kz,kw,B,beta)
    Hdkxdky = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx+dkx,ky+dky,kz,kw,B,beta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkx = np.linalg.eigh(Hdkx)
    _,eigvecdky = np.linalg.eigh(Hdky)
    _,eigvecdkxdky = np.linalg.eigh(Hdkxdky)

    Ux = np.zeros((2,2),dtype=complex)
    Uyx = np.zeros((2,2),dtype=complex)
    Uy = np.zeros((2,2),dtype=complex)
    Uxy = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Ux[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkx[:,j+2].reshape(4,1)))[0][0]
            Uyx[i,j] = (((eigvecdkx[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkxdky[:,j+2].reshape(4,1)))[0][0]
            Uxy[i,j] = (((eigvecdky[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkxdky[:,j+2].reshape(4,1)))[0][0]
            Uy[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdky[:,j+2].reshape(4,1)))[0][0]
    U = Ux@Uyx@np.linalg.inv(Uxy)@np.linalg.inv(Uy)
    return logm(U)

def nonAbeianUzw(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    H =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta)
    Hdkz =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz+dkz,kw,B,beta)
    Hdkw =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw+dkw,B,beta)
    Hdkzdkw =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz+dkz,kw+dkw,B,beta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkz = np.linalg.eigh(Hdkz)
    _,eigvecdkw = np.linalg.eigh(Hdkw)
    _,eigvecdkzdkw = np.linalg.eigh(Hdkzdkw)
    Uz = np.zeros((2,2),dtype=complex)
    Uwz = np.zeros((2,2),dtype=complex)
    Uw = np.zeros((2,2),dtype=complex)
    Uzw = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uz[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkz[:,j+2].reshape(4,1)))[0][0]
            Uwz[i,j] = (((eigvecdkz[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkzdkw[:,j+2].reshape(4,1)))[0][0]
            Uzw[i,j] = (((eigvecdkw[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkzdkw[:,j+2].reshape(4,1)))[0][0]
            Uw[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkw[:,j+2].reshape(4,1)))[0][0]
    U = Uz@Uwz@np.linalg.inv(Uzw)@np.linalg.inv(Uw)
    return logm(U)

def nonAbeianUwx(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    H =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta)
    Hdkx =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx+dkx,ky,kz,kw,B,beta)
    Hdkw = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw+dkw,B,beta)
    Hdkxdkw = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx+dkx,ky,kz,kw+dkw,B,beta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkx = np.linalg.eigh(Hdkx)
    _,eigvecdkw = np.linalg.eigh(Hdkw)
    _,eigvecdkxdkw = np.linalg.eigh(Hdkxdkw)
    Uw = np.zeros((2,2),dtype=complex)
    Uxw = np.zeros((2,2),dtype=complex)
    Uwx = np.zeros((2,2),dtype=complex)
    Ux = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uw[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkw[:,j+2].reshape(4,1)))[0][0]
            Uwx[i,j] = (((eigvecdkx[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkxdkw[:,j+2].reshape(4,1)))[0][0]
            Uxw[i,j] = (((eigvecdkw[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkxdkw[:,j+2].reshape(4,1)))[0][0]
            Ux[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkx[:,j+2].reshape(4,1)))[0][0]
    U = Uw@Uxw@np.linalg.inv(Uwx)@np.linalg.inv(Ux)
    return logm(U)

def nonAbeianUzy(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    H = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta)
    Hdkz = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz+dkz,kw,B,beta)
    Hdky = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky+dky,kz,kw,B,beta)
    Hdkzdky = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky+dky,kz+dkz,kw,B,beta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkz = np.linalg.eigh(Hdkz)
    _,eigvecdky = np.linalg.eigh(Hdky)
    _,eigvecdkzdky = np.linalg.eigh(Hdkzdky)
    Uz = np.zeros((2,2),dtype=complex)
    Uyz = np.zeros((2,2),dtype=complex)
    Uzy = np.zeros((2,2),dtype=complex)
    Uy = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uz[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkz[:,j+2].reshape(4,1)))[0][0]
            Uyz[i,j] = (((eigvecdkz[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkzdky[:,j+2].reshape(4,1)))[0][0]
            Uzy[i,j] = (((eigvecdky[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkzdky[:,j+2].reshape(4,1)))[0][0]
            Uy[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdky[:,j+2].reshape(4,1)))[0][0]
    U = Uz@Uyz@np.linalg.inv(Uzy)@np.linalg.inv(Uy)
    return logm(U)

def nonAbeianUzx(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    H = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta)
    Hdkz = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz+dkz,kw,B,beta)
    Hdkx = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx+dkx,ky,kz,kw,B,beta)
    Hdkxdkz = testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx+dkx,ky,kz+dkz,kw,B,beta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkz = np.linalg.eigh(Hdkz)
    _,eigvecdkx = np.linalg.eigh(Hdkx)
    _,eigvecdkxdkz = np.linalg.eigh(Hdkxdkz)
    Uz = np.zeros((2,2),dtype=complex)
    Uxz = np.zeros((2,2),dtype=complex)
    Uzx = np.zeros((2,2),dtype=complex)
    Ux = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uz[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkz[:,j+2].reshape(4,1)))[0][0]
            Uxz[i,j] = (((eigvecdkz[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkxdkz[:,j+2].reshape(4,1)))[0][0]
            Uzx[i,j] = (((eigvecdkx[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkxdkz[:,j+2].reshape(4,1)))[0][0]
            Ux[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkx[:,j+2].reshape(4,1)))[0][0]
    U = Uz@Uxz@np.linalg.inv(Uzx)@np.linalg.inv(Ux)
    return logm(U)

def nonAbeianUyw(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    H =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta)
    Hdky =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky+dky,kz,kw,B,beta)
    Hdkw =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw+dkw,B,beta)
    Hdkydkw =  testHamiltonian(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky+dky,kz,kw+dkw,B,beta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdky = np.linalg.eigh(Hdky)
    _,eigvecdkw = np.linalg.eigh(Hdkw)
    _,eigvecdkydkw = np.linalg.eigh(Hdkydkw)
    Uy = np.zeros((2,2),dtype=complex)
    Uwy = np.zeros((2,2),dtype=complex)
    Uyw = np.zeros((2,2),dtype=complex)
    Uw = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uy[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdky[:,j+2].reshape(4,1)))[0][0]
            Uwy[i,j] = (((eigvecdky[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkydkw[:,j+2].reshape(4,1)))[0][0]
            Uyw[i,j] = (((eigvecdkw[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkydkw[:,j+2].reshape(4,1)))[0][0]
            Uw[i,j] = (((eigvec[:,i+2].reshape(4,1)).T.conjugate()) @ (eigvecdkw[:,j+2].reshape(4,1)))[0][0]
    U = Uy@Uwy@np.linalg.inv(Uyw)@np.linalg.inv(Uw)
    return logm(U)

def FlP(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw):
    Uxy = nonAbeianUxy(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw)
    Uzw = nonAbeianUzw(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw)
    Uwx = nonAbeianUwx(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw)
    Uzy = nonAbeianUzy(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw)
    Uzx = nonAbeianUzx(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw)
    Uyw = nonAbeianUyw(mz0,mz,t0,tz,tso,tsoz,ton,kx,ky,kz,kw,B,beta,dkx,dky,dkz,dkw)
    return Uxy@Uzw + Uwx@Uzy + Uzx@Uyw
    




In [36]:
t0 = 0.065 #0.0855*scale
tz = 0.097
tso = 0.05 #0.0451*scale
tsoz = 0.05
ton = 0.1 #0.5674*0.15*scale
mz0 = np.array([0.065*7]) #*scale
mz = 0.13 #*scale
N= 20
kx = np.arange(-N,N,2)/N*np.pi
ky = np.arange(-N,N,2)/N*np.pi
kz = np.arange(-N,N,2)/N*np.pi
kw = np.arange(-N,N,2)/N*np.pi
B=0
beta=np.sqrt(2)
Fk = np.zeros((len(kx),len(ky),len(kz),len(kw),2,2),dtype=complex)
dkx = kx[1]-kx[0]
dky = ky[1]-ky[0]
dkz = kz[1]-kz[0]
dkw = kw[1]-kw[0]
Chen2 = np.zeros(len(mz0),dtype=complex)
for mi in np.arange(len(mz0)):
    for i in range(len(kx)):
        for j in range(len(ky)):
            for k in range(len(kz)):
                for l in range(len(kw)):
                    Fk[i,j,k,l] = FlP(mz0[mi],mz,t0,tz,tso,tsoz,ton,kx[i],ky[j],kz[k],kw[l],B,beta,dkx,dky,dkz,dkw)
    Ftrace = np.zeros((len(kx),len(ky),len(kz),len(kw)),dtype=complex)
    for i in range(len(kx)):
        for j in range(len(ky)):
            for k in range(len(kz)):
                for l in range(len(kw)):
                    Ftrace[i,j,k,l] = np.trace(Fk[i,j,k,l])
    Chen2[mi] = 1/(4*np.pi**2)*np.sum(Ftrace)

In [38]:
Chen2

array([-0.92044737+6.42187757e-17j])

In [21]:
Chen2

array([-0.92044737+6.42187757e-17j])